Author: Ronny F. Pacheco Date: Nov 2024 Copyright: © 2024 Ronny Pacheco License: MIT License

MIT License

Copyright (c) 2024 Ronny Pacheco

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


# In/Out files

- In files:
    - ./data/sider_merged.csv
    -  ../data/bringaud_siders.csv
- Out files:
    - ./data/bedops/compare (directory) 

# Needed modules

In [1]:
import pandas as pd
import pickle
import os
import subprocess

from jupyter_core.version import pattern

In [2]:
# https://kioku-space.com/en/jupyter-skip-execution/
from IPython.core.magic import register_cell_magic # type: ignore


@register_cell_magic
def skip(line, cell):
    print('Skipping cell')
    if line and cell:
        pass
    return

# Pickle save

In [3]:
%%skip
# =============================================================================
# main function
# =============================================================================
def data_save_load(option):
    """
    This function is used to save or load 0.data for the jupyter notebook
    """
    path_folder = "ipynb_db"  # Folder to save variables
    os.makedirs(path_folder, exist_ok=True)  # Create folder if not exist
    notebook_name = '1.prepare_data'
    path_file = os.path.join(path_folder, f"{notebook_name}.variables.pkl") # Path to save the variables

    if option == "save":
        with open(path_file, "wb") as pickle_file:
            dict_variables = { # 'var_name' : 'real_var'

            }
            pickle.dump(dict_variables, pickle_file)
    elif option == "load":
        with open(path_file, "rb") as pickle_file:
            variables = pickle.load(pickle_file)
        # Now load the variables
        for pickle_key, pickle_value in variables.items():
            print(f"* Loading variable: {pickle_key}")
            globals()[pickle_key] = pickle_value
# =============================================================================
# Call the function
# =============================================================================
data_save_load(option="load")

Skipping cell


# Load data

## SIDER form our software

In [4]:
# Load sider merged 0.data
sider_df = pd.read_csv('./data/sider_merged.csv', sep=',', header=0)
print(sider_df.shape)
print(sider_df.dtypes)
sider_df.head()

(2874, 5)
chrom          object
sider_start     int64
sider_end       int64
strand         object
sider_seq      object
dtype: object


,chrom,sider_start,sider_end,strand,sider_seq
0,LmjF.01,24284,24536,plus,GCAGCCCCCTCCCCCTCTATCCCTGCCAACGCCAAACCACTTCTGG...
1,LmjF.01,24604,25200,plus,CCGGCATGATGGAGGGGCGGCTGTGAGGTGGTGCGTGCATGTATGT...
2,LmjF.01,35467,36401,plus,CCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACACGCGCTGCAG...
3,LmjF.01,39730,40541,plus,ATTTTACCGCGAGCAAGGCAACACACAGACGCACACACAACCACAG...
4,LmjF.01,54594,55542,plus,GCCGCACCCCACCCCCATCGCCTGGTGCGAAGCAGCGCAAGACACA...


## Bringaud SIDER

In [5]:
# Load bringaud sider 0.data
bringaud_df = pd.read_csv('../0.data/bringaud_siders.csv', sep=',', header=0)
print(bringaud_df.shape)
print(bringaud_df.dtypes)
bringaud_df.head()

(1852, 8)
ID          object
chr          int64
name        object
fam          int64
str         object
start        int64
end          int64
chr_size     int64
dtype: object


,ID,chr,name,fam,str,start,end,chr_size
0,51.te00022,1,chr1_37.00281,1,-,39705,40532,268984
1,51.te00019,1,chr1_37.00289,1,+,260993,261261,268984
2,51.te00020,1,chr1_37.00290,1,+,266162,266340,268984
3,51.te00018,1,chr1_37.00279,2,+,24174,24821,268984
4,51.te00021,1,chr1_37.00280,2,-,35328,36036,268984


In [6]:
bringaud_df_test = bringaud_df.copy()

In [7]:
bringaud_df_chrom = bringaud_df_test[['name', 'chr_size']].copy()
bringaud_df_chrom['name'] = bringaud_df_chrom['name'].str.extract(r'(^\D+\d+)_', expand=True)
bringaud_df_chrom.head()

,name,chr_size
0,chr1,268984
1,chr1,268984
2,chr1,268984
3,chr1,268984
4,chr1,268984


In [8]:
# Extract the 'chr' number and convert them to int for sorting
bringaud_df_chrom['chr_number'] = bringaud_df_chrom['name'].str.extract(r'(\d+)', expand=False).astype(int)

# Sort the DataFrame by the extracted 'chr_number'
bringaud_chrom_count = pd.DataFrame(bringaud_df_chrom.value_counts())

bringaud_df_chrom.drop_duplicates(subset=['name'], inplace=True)
bringaud_df_chrom

,name,chr_size,chr_number
0,chr1,268984,1
12,chr2,355714,2
20,chr3,384518,3
27,chr4,472856,4
40,chr5,465823,5
59,chr6,513422,6
84,chr7,596348,7
110,chr8,559245,8
148,chr9,573441,9
170,chr10,570864,10


# Prepare data

## Our SIDERs

In [9]:
# Common names and only needed columns
sider_df = sider_df[['chrom', 'sider_start', 'sider_end']]
sider_df = sider_df.rename(columns={'chrom' : 'chrom', 'sider_start' : 'start', 'sider_end' : 'end'})
sider_df.head()

,chrom,start,end
0,LmjF.01,24284,24536
1,LmjF.01,24604,25200
2,LmjF.01,35467,36401
3,LmjF.01,39730,40541
4,LmjF.01,54594,55542


## Bringaud siders

In [10]:
# Take from bringaud only the needed 0.data
bringaud_df = bringaud_df[['name', 'start', 'end']]
bringaud_df = bringaud_df.rename(columns={'name' : 'chrom'})
bringaud_df.head()

,chrom,start,end
0,chr1_37.00281,39705,40532
1,chr1_37.00289,260993,261261
2,chr1_37.00290,266162,266340
3,chr1_37.00279,24174,24821
4,chr1_37.00280,35328,36036


In [11]:
# Get the chromosome and prepend '0' to single digits, then add 'LmJF.'
pattern = r'^\D+(\d+)_'
bringaud_df['chrom'] = bringaud_df['chrom'].str.extract(pattern, expand=True)

# Now for the numbers like 0, 1, 2.. get 01, 02, 03...
bringaud_df['chrom'] = 'LmjF.' + bringaud_df['chrom'].str.zfill(2)
bringaud_df.head()

,chrom,start,end
0,LmjF.01,39705,40532
1,LmjF.01,260993,261261
2,LmjF.01,266162,266340
3,LmjF.01,24174,24821
4,LmjF.01,35328,36036


In [12]:
# Check if 'bringaud_start' > 'bringaud_end'. It should be 0
bringaud_df[bringaud_df['start'] > bringaud_df['end']].shape

(0, 3)

# Compare

## Needed functions

In [13]:
# Let's define the function since it seems that we will use it more than once
def compare_sequences(df_group1, df_group2, df_group1_og_data, path):
    """
    
    :param df_group1: True Positive 0.data, what we want to get.
    :param df_group2: 
    :param df_group1_og_data: 
    :param path: 
    :return: 
    """
    compare_dict = {}
    total = 0
    not_captured_df = pd.DataFrame()
    
    for (name1, group1), (name2, group2) in zip(df_group1, df_group2):
        # group1 should be True Positive 0.data
        # ------------------------------------------------------------------------------
        path_chr = os.path.join(path, name1)
        os.makedirs(path_chr, exist_ok=True)
        # ------------------------------------------------------------------------------
        group1_total = group1[["chrom", "start", "end"]].copy()
        group2_total = group2[["chrom", "start", "end"]].copy()
        # ------------------------------------------------------------------------------
        group1_total.sort_values(by=["start", "end"], inplace=True)
        group2_total.sort_values(by=["start", "end"], inplace=True)
        # ------------------------------------------------------------------------------
        path_group1_total = os.path.join(path_chr, "group1_total.bed")
        path_group2_total = os.path.join(path_chr, "group2_total.bed")
        # ------------------------------------------------------------------------------
        group1_total.to_csv(path_group1_total, sep="\t", header=False, index=False)  # tabular sep because of bed format
        group2_total.to_csv(path_group2_total, sep="\t", header=False, index=False)  # tabular sep because of bed format

        # ------------------------------------------------------------------------------
        condition = 1
        cmd = f"bedops --element-of {condition} {path_group1_total} {path_group2_total}"
        result = subprocess.check_output(cmd, shell=True, universal_newlines=True)
        result_df = pd.DataFrame([x.split("\t") for x in result.split("\n") if x], 
                                    columns=["chrom", "start", "end"])
        result_df[['start', 'end']] = result_df[['start', 'end']].apply(pd.to_numeric)
        compare_dict[name1] = [f"{result_df.shape[0]}/{group1_total.shape[0]}", f"{result_df.shape[0]/group1_total.shape[0]*100:.2f} %"]
        total += result_df.shape[0]
        
        # ------------------------------------------------------------------------------
        not_cmd = f"bedops --not-element-of {condition} {path_group1_total} {path_group2_total}"
        not_result = subprocess.check_output(not_cmd, shell=True, universal_newlines=True)
        not_result_df = pd.DataFrame([x.split("\t") for x in not_result.split("\n") if x], 
                                    columns=["chrom", "start", "end"])
        not_result_df[['start', 'end']] = not_result_df[['start', 'end']].apply(pd.to_numeric)
        not_captured_df = pd.concat([not_captured_df, not_result_df])
        
    print(f"""
    There are {total} first DF sequences of {df_group1_og_data.shape[0]} in the second DF:
        - That's {total / df_group1_og_data.shape[0] * 100:.2f}% of the TP 0.data.
        - {df_group1_og_data.shape[0] - total} TP sequences are not in consensus 0.data, 
          which is {100 - total / df_group1_og_data.shape[0] * 100:.2f}%.
    """)
    return compare_dict, not_captured_df

## Prepare folder env

In [14]:
folder_path = './data/bedops/compare'
os.makedirs(folder_path, exist_ok=True)

In [15]:
sider_df_grouped = sider_df.groupby('chrom')
bringaud_df_grouped = bringaud_df.groupby('chrom')

## Comparison

In [16]:
sider_vs_bringaud, not_captured = compare_sequences(df_group1=bringaud_df_grouped,
                                                    df_group2=sider_df_grouped,
                                                    df_group1_og_data=bringaud_df,
                                                    path=folder_path)
sider_vs_bringaud


    There are 1323 first DF sequences of 1852 in the second DF:
        - That's 71.44% of the TP data.
        - 529 TP sequences are not in consensus data, 
          which is 28.56%.
    


{'LmjF.01': ['11/12', '91.67 %'],
 'LmjF.02': ['8/8', '100.00 %'],
 'LmjF.03': ['7/7', '100.00 %'],
 'LmjF.04': ['12/13', '92.31 %'],
 'LmjF.05': ['17/19', '89.47 %'],
 'LmjF.06': ['6/25', '24.00 %'],
 'LmjF.07': ['26/26', '100.00 %'],
 'LmjF.08': ['33/38', '86.84 %'],
 'LmjF.09': ['16/22', '72.73 %'],
 'LmjF.10': ['24/25', '96.00 %'],
 'LmjF.11': ['26/28', '92.86 %'],
 'LmjF.12': ['12/33', '36.36 %'],
 'LmjF.13': ['26/30', '86.67 %'],
 'LmjF.14': ['32/36', '88.89 %'],
 'LmjF.15': ['20/22', '90.91 %'],
 'LmjF.16': ['10/27', '37.04 %'],
 'LmjF.17': ['2/41', '4.88 %'],
 'LmjF.18': ['3/60', '5.00 %'],
 'LmjF.19': ['46/51', '90.20 %'],
 'LmjF.20': ['4/39', '10.26 %'],
 'LmjF.21': ['5/34', '14.71 %'],
 'LmjF.22': ['42/50', '84.00 %'],
 'LmjF.23': ['26/54', '48.15 %'],
 'LmjF.24': ['41/43', '95.35 %'],
 'LmjF.25': ['31/34', '91.18 %'],
 'LmjF.26': ['46/56', '82.14 %'],
 'LmjF.27': ['57/59', '96.61 %'],
 'LmjF.28': ['3/49', '6.12 %'],
 'LmjF.29': ['64/91', '70.33 %'],
 'LmjF.30': ['82/91', '9

In [17]:
# not_captured.to_csv('./0.data/bedops/not_captured.csv', header=True, sep=',')
not_captured.head()

,chrom,start,end
0,LmjF.01,226657,226940
0,LmjF.04,181541,181927
0,LmjF.05,90841,91072
1,LmjF.05,464001,464509
0,LmjF.06,57422,57917
